In [1]:
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipysheet import from_dataframe
from src.post_processing import PathWrangler
import pandas as pd

In [2]:
path_filepath = '../artifacts/processed_expansions/found_paths.json'
predicted_reactions_filepath = "../artifacts/processed_expansions/predicted_reactions.json"
known_reactions_filepath = "../artifacts/processed_expansions/known_reactions.json"

pw = PathWrangler(
    path_filepath=path_filepath,
    pr_filepath=predicted_reactions_filepath,
    kr_filepath=known_reactions_filepath
)

In [3]:
# Options
starter_options, target_options = zip(*pw.starter_targets)
target_options = tuple(set(target_options))
evidence_options = [(elt.value, elt.name) for elt in pw.enzyme_existence]
sort_options = [
    ("Mean RCMCS", 'mean_rcmcs'),
    ("Min RCMCS", 'min_rcmcs'),
    ("Max-min driving force", 'mdf')
]

# (Default) selection
starter_selection = starter_options[:3]
target_selection = target_options[:3]
loe_selection = [pw.enzyme_existence.PROTEIN.name]
sort_by_selection = 'mean_rcmcs'

# Define filter / sort widgets
starter_selector = widgets.SelectMultiple(
    options=starter_options,
    value=starter_selection,
    rows=len(starter_options),
    description='Starters:',
    disabled=False,
    continuous_update=False
)

target_selector = widgets.SelectMultiple(
    options=target_options,
    value=target_selection,
    rows=len(target_options),
    description='Targets:',
    disabled=False
)

evidence_selector = widgets.SelectMultiple(
    options=evidence_options,
    value=loe_selection,
    rows=len(evidence_options),
    description='Enzyme level of evidence:',
    disabled=False
)

sort_by_radio_buttons = widgets.RadioButtons(
    options=sort_options,
    value=sort_by_selection,
    description='Sort paths by:',
    disabled=False
)

selectors = widgets.HBox(
    [
        starter_selector,
        target_selector,
        evidence_selector,
        sort_by_radio_buttons
    ]
)

# Output contexts
gen_paths_out = widgets.Output()
path_viewer_out = widgets.Output()

# Define callbacks
def generate_paths():
    global paths
    filter_by_enzymes = {'existence': loe_selection} # Only paths w/ enzymes w/ protein-level evidence
    paths = pw.get_paths(
        starters=starter_selection,
        targets=target_selection,
        sort_by=sort_by_selection,
        filter_by_enzymes=filter_by_enzymes
    )

    with gen_paths_out:
        clear_output()
        print(f"{len(paths)} paths meet your criteria")

    with path_viewer_out:
        clear_output()
        print(f"Loading paths...")

    update_path_selector()
    update_path_viewer()
    redisplay_paths()

def starter_listener(change):
    global starter_selection
    starter_selection = starter_selector.value
    generate_paths()

def target_listener(change):
    global target_selection
    target_selection = target_selector.value
    generate_paths()

def evidence_listener(change):
    global loe_selection
    loe_selection = [evidence_options[k] for k in evidence_selector.value]
    generate_paths()

def sort_by_listener(change):
    global sort_by_selection
    sort_by_selection = sort_by_radio_buttons.value
    generate_paths()

def update_path_selector():
    global path_selector
    value = 0 if paths else None
    path_selector = widgets.Dropdown(
        options=[(i+1, i) for i in range(len(paths))],
        value=value,
        description='Path #',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )

def display_enzymes(enzymes):
    df = pd.DataFrame(data=[e.to_dict() for e in enzymes])
    df = df.loc[:, ['uniprot_id', 'ec', 'organism', 'name', 'existence', 'reviewed', 'sequence']]

    return from_dataframe(df)

def display_predicted_reaction(rxn_step, img):
    html = f"""
    <b><u>Step #{rxn_step+1}</u></b>
    <div>
        <img src="{img}" style="width:950px">
    </div>
    """
    return widgets.HTML(html)

def display_analogue(img, rcmcs):
    html = f"""
    <b><u>{rcmcs * 100:.2f}% similar to predicted reaction</u></b>
    <div>
        <img src="{img}" style="width:900px">
    </div>
    """
    return widgets.HTML(value=html)

def display_analogues_enzymes(krs, rcmcses):
    kr_elts = []
    enzyme_elts = []
    for kr, rcmcs in zip(krs, rcmcses):
        kr_elts.append(display_analogue(kr.image, rcmcs))
        enzyme_elts.append(display_enzymes(kr.enzymes))

    kr_selector = widgets.Dropdown(
        options=[(i+1, i) for i in range(len(kr_elts))],
        value=0,
        description="Known analogue: "
    )
    kr_stack = widgets.Stack(kr_elts, selected_index=0)
    kr_sel_disp = widgets.VBox([kr_selector, kr_stack])
    enzyme_stack = widgets.Stack(enzyme_elts, selected_index=0)
    link_kr = widgets.jslink((kr_selector, 'index'), (kr_stack, 'selected_index'))
    link_enz = widgets.jslink((kr_selector, 'index'), (enzyme_stack, 'selected_index'))

    tab_titles = ['Known Analogue', 'Enzymes']
    tab_children = [kr_sel_disp, enzyme_stack]
    tab = widgets.Tab(
        children=tab_children,
        titles=tab_titles,
        layout=widgets.Layout(width="950px")
    )

    return tab

def display_path(path_idx):
    k = 1000
    path = paths[path_idx]
    header = f"""
    <h3>{len(path.reactions)}-step path from {path.starter.upper()} to {path.target.upper()}<br>
    Max-min driving force = {path.mdf:.2f} kJ/mol<br>
    </h3>
    """
    rows = [widgets.HTML(value=header)]
    for i, reaction in enumerate(path.reactions):
        pr_image = reaction.image
        analogues = reaction.top_analogues(k=k)
        rcmcses = reaction.top_rcmcs(k=k)
        pr_elt = display_predicted_reaction(i, pr_image)
        kr_elt = display_analogues_enzymes(analogues, rcmcses)
        rows.append(widgets.HBox([pr_elt, kr_elt], layout=widgets.Layout(border= '1px solid black')))

    return widgets.VBox(rows)

def update_path_viewer():
    global test_stack, test_link
    layers = [display_path(i) for i in range(len(paths))]
    test_stack = widgets.Stack(layers, selected_index=None)
    test_link = widgets.jslink((path_selector, 'index'), (test_stack, 'selected_index'))

def redisplay_paths():
    with path_viewer_out:
        clear_output()
        display(path_selector, test_stack)

# Attach listeners to UI widgets
starter_selector.observe(starter_listener, names=['value'])
target_selector.observe(target_listener, names=['value'])
evidence_selector.observe(evidence_listener, names=['value'])
sort_by_radio_buttons.observe(sort_by_listener, names=['value'])

# Initialize paths
generate_paths()

In [4]:
display(selectors)
display(gen_paths_out)

Output(outputs=({'name': 'stdout', 'text': '25 paths meet your criteria\n', 'output_type': 'stream'},))

In [5]:
display(path_viewer_out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "Dropdown(description='Path #', options…

Predicted pathway viewer